# Inference with NLI validation

In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import re

In [2]:
prefix = "https://huggingface.co/muhammadravi251001/fine-tuned-FilteringNLI-"
suffix = "/blob/main/results/evaluation/metric_result.txt"

In [3]:
def em_and_f1(p_url, acc, id="L1"):

    page = requests.get(p_url)
    soup = BeautifulSoup(page.content)

    tr = soup.find('tr', attrs={'id': id})
    td = tr.find(attrs={'class': 'overflow-visible whitespace-pre px-3'}).text
    x = re.findall("\d+\.\d+", td)

    if acc == True: return round(float(x[0]), 3)
    else: return round(float(x[1]), 3)

In [4]:
from collections import OrderedDict

def take_all_url(data):

    baseline = f"{prefix}indonli_mnli-{data}-TQ2-TS4-MS3-VA0-TH0.0{suffix}"
    msc_arr = ["indonli", "indonli_mnli", f"indonli_mnli_{data}-nli"]
    tq_arr = [1, 2]
    msi_arr = [2, 3]
    var_arr = [1, 2, 3]
    th_arr = [0.25, 0.5, 0.75]

    url_arr = []
    url_arr.append(baseline)

    for msc in msc_arr:
        for tq in tq_arr:
            for msi in msi_arr:
                for var in var_arr:
                    for th in th_arr:
                        
                        if msc == "indonli" or msc == "indonli_mnli":
                            var = 1
                            th = 0.0

                        if var == 1:
                            th = 0.0

                        url_arr.append(f"{prefix}{msc}-{data}-TQ{tq}-TS4-MS{msi}-VA{var}-TH{th}{suffix}")
    
    url_arr = list(OrderedDict.fromkeys(url_arr))
    print("Amount of url:", len(url_arr))
    return url_arr

In [21]:
def extract_values(url, data):

    msc_list = f"indonli_mnli_{data}-nli|indonli_mnli|indonli"
    pattern = re.compile(r'.*FilteringNLI-({})-{}-TQ(\d+)-TS(\d+)-MS([\d]+)-VA([\d]+)-TH([\d.]+).*'.format(msc_list, data))
    match = pattern.match(url)
    
    if match:
        msc = match.group(1)
        TQ = int(match.group(2))
        TS = int(match.group(3))
        MSI = int(match.group(4))
        VA = int(match.group(5))
        
        TH = float(match.group(6))
        if TH == '0.00':
            TH = '0.0'
        elif TH == '0.50':
            TH = '0.5'

        EM = em_and_f1(url, True)
        F1 = em_and_f1(url, False)

        return msc, data, TQ, TS, MSI, VA, TH, EM, F1
    
    else:
        return None

In [6]:
import pandas as pd
import re
from tqdm import tqdm

def create_table(url_list, data):
    
    data_list = []
    for url in tqdm(url_list, desc="Processing URLs", unit="URL"):
        extracted_values = extract_values(url, data)
        data_list.append(extracted_values)

    df = pd.DataFrame(data_list, columns=['MSC', 'data', 'TQ', 'TS', 'MSI', 'VA', 'TH', 'EM', 'F1'])
    
    df.loc[0, 'MSC'] = "Baseline"
    df.loc[0, 'TQ'] = "BL"
    df.loc[0, 'TS'] = "BL"
    df.loc[0, 'MSI'] = "BL"
    df.loc[0, 'VA'] = "BL"
    df.loc[0, 'TH'] = "BL"
    
    return df

# IDK-MRC

In [7]:
data = "idkmrc"
url_list = take_all_url(data)
table = pd.DataFrame(create_table(url_list, data))
table.to_excel(f'{data}-nli-validator.xlsx', index=False)
table

Amount of url: 37


Processing URLs: 100%|█████████████████████████████████████████████████████████████████| 37/37 [00:31<00:00,  1.18URL/s]


,MSC,data,TQ,TS,MSI,VA,TH,EM,F1
0,Baseline,idkmrc,BL,BL,BL,BL,BL,75.829,84.251
1,indonli,idkmrc,1,4,2,1,0.0,54.621,62.089
2,indonli,idkmrc,1,4,3,1,0.0,43.957,51.957
3,indonli,idkmrc,2,4,2,1,0.0,73.934,82.222
4,indonli,idkmrc,2,4,3,1,0.0,72.630,80.936
5,indonli_mnli,idkmrc,1,4,2,1,0.0,61.967,69.509
6,indonli_mnli,idkmrc,1,4,3,1,0.0,52.607,60.546
7,indonli_mnli,idkmrc,2,4,2,1,0.0,72.630,81.104
8,indonli_mnli,idkmrc,2,4,3,1,0.0,71.445,79.920
9,indonli_mnli_idkmrc-nli,idkmrc,1,4,2,1,0.0,61.493,70.766


In [9]:
sorted_table = table.sort_values(by='F1', ascending=False)
sorted_table

,MSC,data,TQ,TS,MSI,VA,TH,EM,F1
0,Baseline,idkmrc,BL,BL,BL,BL,BL,75.829,84.251
3,indonli,idkmrc,2,4,2,1,0.0,73.934,82.222
7,indonli_mnli,idkmrc,2,4,2,1,0.0,72.630,81.104
4,indonli,idkmrc,2,4,3,1,0.0,72.630,80.936
8,indonli_mnli,idkmrc,2,4,3,1,0.0,71.445,79.920
23,indonli_mnli_idkmrc-nli,idkmrc,2,4,2,1,0.0,62.441,71.713
24,indonli_mnli_idkmrc-nli,idkmrc,2,4,2,2,0.25,62.441,71.713
25,indonli_mnli_idkmrc-nli,idkmrc,2,4,2,2,0.5,61.730,71.003
9,indonli_mnli_idkmrc-nli,idkmrc,1,4,2,1,0.0,61.493,70.766
10,indonli_mnli_idkmrc-nli,idkmrc,1,4,2,2,0.25,61.493,70.766


# TYDI-QA-ID

In [23]:
data = "tydiqaid"
url_list = take_all_url(data)
table = pd.DataFrame(create_table(url_list, data))
table.to_excel(f'{data}-nli-validator.xlsx', index=False)
table

Amount of url: 37


Processing URLs: 100%|█████████████████████████████████████████████████████████████████| 37/37 [00:34<00:00,  1.07URL/s]


,MSC,data,TQ,TS,MSI,VA,TH,EM,F1
0,Baseline,tydiqaid,BL,BL,BL,BL,BL,61.287,76.476
1,indonli,tydiqaid,1,4,2,1,0.0,36.140,48.788
2,indonli,tydiqaid,1,4,3,1,0.0,36.257,50.225
3,indonli,tydiqaid,2,4,2,1,0.0,60.000,75.234
4,indonli,tydiqaid,2,4,3,1,0.0,60.117,75.353
5,indonli_mnli,tydiqaid,1,4,2,1,0.0,43.509,56.567
6,indonli_mnli,tydiqaid,1,4,3,1,0.0,43.509,57.579
7,indonli_mnli,tydiqaid,2,4,2,1,0.0,60.000,75.278
8,indonli_mnli,tydiqaid,2,4,3,1,0.0,60.000,75.305
9,indonli_mnli_tydiqaid-nli,tydiqaid,1,4,2,1,0.0,61.287,77.310


In [24]:
sorted_table = table.sort_values(by='F1', ascending=False)
sorted_table

,MSC,data,TQ,TS,MSI,VA,TH,EM,F1
14,indonli_mnli_tydiqaid-nli,tydiqaid,1,4,2,3,0.5,61.287,77.453
28,indonli_mnli_tydiqaid-nli,tydiqaid,2,4,2,3,0.5,61.287,77.453
27,indonli_mnli_tydiqaid-nli,tydiqaid,2,4,2,3,0.25,61.287,77.453
13,indonli_mnli_tydiqaid-nli,tydiqaid,1,4,2,3,0.25,61.287,77.453
29,indonli_mnli_tydiqaid-nli,tydiqaid,2,4,2,3,0.75,61.170,77.426
15,indonli_mnli_tydiqaid-nli,tydiqaid,1,4,2,3,0.75,61.170,77.426
21,indonli_mnli_tydiqaid-nli,tydiqaid,1,4,3,3,0.5,61.287,77.336
35,indonli_mnli_tydiqaid-nli,tydiqaid,2,4,3,3,0.5,61.287,77.336
34,indonli_mnli_tydiqaid-nli,tydiqaid,2,4,3,3,0.25,61.287,77.336
20,indonli_mnli_tydiqaid-nli,tydiqaid,1,4,3,3,0.25,61.287,77.336


# SQUAD-ID

In [25]:
data = "squadid"
url_list = take_all_url(data)
table = pd.DataFrame(create_table(url_list, data))
table.to_excel(f'{data}-nli-validator.xlsx', index=False)
table

Amount of url: 37


Processing URLs: 100%|█████████████████████████████████████████████████████████████████| 37/37 [00:33<00:00,  1.12URL/s]


,MSC,data,TQ,TS,MSI,VA,TH,EM,F1
0,Baseline,squadid,BL,BL,BL,BL,BL,47.915,67.076
1,indonli,squadid,1,4,2,1,0.0,13.788,23.243
2,indonli,squadid,1,4,3,1,0.0,13.796,25.484
3,indonli,squadid,2,4,2,1,0.0,40.958,57.654
4,indonli,squadid,2,4,3,1,0.0,40.958,58.033
5,indonli_mnli,squadid,1,4,2,1,0.0,22.025,34.017
6,indonli_mnli,squadid,1,4,3,1,0.0,22.058,35.855
7,indonli_mnli,squadid,2,4,2,1,0.0,41.826,58.721
8,indonli_mnli,squadid,2,4,3,1,0.0,41.826,59.046
9,indonli_mnli_squadid-nli,squadid,1,4,2,1,0.0,47.764,66.990


In [26]:
sorted_table = table.sort_values(by='F1', ascending=False)
sorted_table

,MSC,data,TQ,TS,MSI,VA,TH,EM,F1
0,Baseline,squadid,BL,BL,BL,BL,BL,47.915,67.076
28,indonli_mnli_squadid-nli,squadid,2,4,2,3,0.5,47.806,67.054
27,indonli_mnli_squadid-nli,squadid,2,4,2,3,0.25,47.806,67.054
13,indonli_mnli_squadid-nli,squadid,1,4,2,3,0.25,47.806,67.053
14,indonli_mnli_squadid-nli,squadid,1,4,2,3,0.5,47.806,67.053
21,indonli_mnli_squadid-nli,squadid,1,4,3,3,0.5,47.781,67.041
20,indonli_mnli_squadid-nli,squadid,1,4,3,3,0.25,47.781,67.041
35,indonli_mnli_squadid-nli,squadid,2,4,3,3,0.5,47.781,67.038
34,indonli_mnli_squadid-nli,squadid,2,4,3,3,0.25,47.781,67.038
17,indonli_mnli_squadid-nli,squadid,1,4,3,2,0.25,47.764,67.016
